## Business Analytics Platform Project v2.0


### 1. Data Preparation 
####  Yelp DataSet Overview 
##### DataSet Link : https://www.yelp.com/dataset
- Data Format : JSON Files, include bussines.json, checkin.json, review.json, user.json,tip.json, photo.json
- Detailed JSON Format can be found here :https://www.yelp.com/dataset/documentation/main
- 1,223,094 tips by 1,637,138 users
- Over 1.2 million business attributes like hours, parking, availability, and ambience
- Aggregated check-ins over time for each of the 192,609 businesses
- 8.96 GB business data
- Around 8 GB photo zip files

#### Business Data Format Transformation

- Create a python script transform JSON file into multiple CSV files

In [ ]:
import argparse
import collections
import csv
import json


def read_and_write_file(json_file_path, csv_file_path, column_names):
    with open(csv_file_path, 'w+') as fout:
        csv_file = csv.writer(fout)
    csv_file.writerow(list(column_names))
    with open(json_file_path, encoding="utf8") as fin:

        for line in fin:
            line_contents = json.loads(line)
            csv_file.writerow(get_row(line_contents, column_names))


def get_superset_of_column_names_from_file(json_file_path):
    column_names = set()
    with open(json_file_path) as fin:

        for line in fin:
            line_contents = json.loads(line)
        column_names.update(
            set(get_column_names(line_contents).keys())
            )
        return column_names


def get_column_names(line_contents, parent_key=''):
    column_names = []
    for k, v in line_contents.items():
        column_name = "{0}.{1}".format(parent_key, k) if parent_key else k
    if isinstance(v, collections.MutableMapping):
        column_names.extend(
                get_column_names(v, column_name).items()
                )
    else:
        column_names.append((column_name, v))
    return dict(column_names)


def get_nested_value(d, key):

    if '.' not in key:
        if key not in d:
            return None
    return d[key]
    base_key, sub_key = key.split('.', 1)
    if base_key not in d:
        return None
    sub_dict = d[base_key]
    return get_nested_value(sub_dict, sub_key)


def get_row(line_contents, column_names):
    row = []
    for column_name in column_names:
        line_value = get_nested_value(
            line_contents,
            column_name,
            )
    if isinstance(line_value, str):
        row.append('{0}'.format(line_value.encode('utf-8')))
    elif line_value is not None:
        row.append('{0}'.format(line_value))
    else:
        row.append('')
    return row

if __name__ == '__main__':
        parser = argparse.ArgumentParser(
        description='Convert Yelp data from JSON format to CSV.',
        )

        parser.add_argument(
            'json_file',
            type=str,
            help='The json file to convert.',
            )

        args = parser.parse_args()

        json_file = args.json_file
        csv_file = '{0}.csv'.format(json_file.split('.json')[0])

        column_names = get_superset_of_column_names_from_file(json_file)
        read_and_write_file(json_file, csv_file, column_names)

## 2. Data Extract,Transform and Load(ETL)

### - Business DataSet


In [1]:
import pandas as pd



In [2]:
business_df = pd.read_csv('./DataSet/business.csv',low_memory=False)
    
business_df.head()
    

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,hours.Tuesday,hours.Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,9:0-0:0,9:0-0:0,1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15",NaN,NaN,u'beer_and_wine',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,True,NaN,...,NaN,17:30-21:30,1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8:0-17:0,8:0-17:0,1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,...,7:0-23:0,7:0-23:0,1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


### Checkin DataSet


In [3]:
checkin_df = pd.read_csv('./DataSet/checkin.csv')

checkin_df.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [4]:
checkin_df.count()

business_id    161950
date           161950
dtype: int64

In [5]:
checkin_df.columns = ['business_id','checkin_date']
checkin_df.head()

,business_id,checkin_date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


### Tip Dataset


In [6]:
tip_df = pd.read_csv('./DataSet/tip.csv')
tip_df.head()

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


In [7]:
tip_df.count()

business_id         1223094
compliment_count    1223094
date                1223094
text                1223090
user_id             1223094
dtype: int64

In [39]:
tip_df.columns = ['business_id','tip_compliment_count','tip_date','tip_text','user_id']
tip_df.head()

,business_id,tip_compliment_count,tip_date,tip_text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


###  User Dataset


In [9]:
user_df = pd.read_csv('./DataSet/user.csv',low_memory=False)
user_df.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.03,1,0,1,2,0,0,1,0,1,...,25,"2015,2016,2017",5,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",17,Rashmi,95,84,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33
1,3.63,1,0,1,1,0,0,0,0,0,...,16,NaN,4,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",22,Jenna,33,48,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06
2,3.71,0,0,0,0,0,0,1,0,0,...,10,NaN,0,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",8,David,16,28,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10
3,4.85,0,0,0,1,0,0,0,0,2,...,14,NaN,5,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",4,Angela,17,30,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30
4,4.08,80,0,80,28,1,1,16,5,57,...,665,"2015,2016,2017,2018",39,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",279,Nancy,361,1114,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50


### Review Dataset

In [10]:
review_df = pd.read_csv('./DataSet/review.csv')
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1.0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6.0,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0.0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0.0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,ssoyf2_x0EQMed6fgHeMyQ


In [11]:
review_df.count()

business_id    6685902
cool           6685902
date           6685902
funny          6685900
review_id      6685900
stars          6685900
text           6685898
useful         6685898
user_id        6685898
dtype: int64

### Photo Dataset


In [12]:
photo_df = pd.read_csv('./DataSet/photo.csv')
photo_df.head()

,business_id,caption,label,photo_id
0,rcaPajgKOJC2vo_l3xa42A,NaN,inside,MllA1nNpcp1kDteVg6OGUw
1,Kn23LDd740SBVJ7mum0fwg,NaN,inside,YjxBE88Bf6CmTEF2LP1UNA
2,ZkGDCVKSdf8m76cnnalL-A,NaN,food,1f7izSjM0WjkDRIVbPy1yw
3,bF8gv7k_rwZtiDLP2ZB04w,NaN,inside,NcSlcDTEEeOaixotOPk-rA
4,50Anorn0DJXFhBr9a9_gHQ,NaN,inside,5IiIo5UKEW0lWqZ6sWrY_A


### Choose Ontario Business Data as Example
#### Extract seperate dataset

In [32]:
business_ON_df = business_df[business_df['state'] == 'ON']
business_ON_checkin_df = pd.merge(business_ON_df,checkin_df,on='business_id',how='left')
business_ON_checkin_df.head()

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,hours.Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state,checkin_date
0,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,9:0-0:0,1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"2010-07-05 16:56:31, 2010-12-21 20:30:01, 2010..."
1,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9:0-20:0,1,43.599475,-79.711584,The UPS Store,L5V 0B1,3,2.5,ON,"2014-03-31 21:46:17, 2015-01-27 19:05:03, 2018..."
2,1455 16th Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,16:0-23:0,1,43.861503,-79.388499,Hot Yoga Wellness,L4B 3G6,4,4.0,ON,"2016-04-19 17:38:29, 2016-04-22 01:14:54, 2016..."
3,"445 King Street W, Suite 101",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6:0-22:0,1,43.645041,-79.395799,Totum Life Science,M5V 1K4,23,4.0,ON,"2010-10-23 14:00:26, 2010-10-24 16:33:33, 2010..."
4,"865 York Mills Road, Unit 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,9:0-20:0,1,43.754093,-79.349548,Bakery Gateau,M3B 1Y6,8,4.5,ON,"2016-01-24 00:15:08, 2016-01-25 22:42:18, 2016..."


In [34]:
ON_business_id_df = business_ON_checkin_df[['business_id']]
ON_business_id_df.count()

business_id    33412
dtype: int64

In [41]:
ON_biztips_df = pd.merge(ON_business_id_df,tip_df,on='business_id',how='left')
ON_biztips_df.count()

business_id             129212
tip_compliment_count    117518
tip_date                117518
tip_text                117517
user_id                 117518
dtype: int64

In [42]:
# Drop rows with missing values and drop duplicate
ON_biztips_df.dropna(inplace=True)
ON_biztips_df.drop_duplicates(inplace=True)

ON_biztips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117500 entries, 0 to 129211
Data columns (total 5 columns):
business_id             117500 non-null object
tip_compliment_count    117500 non-null float64
tip_date                117500 non-null object
tip_text                117500 non-null object
user_id                 117500 non-null object
dtypes: float64(1), object(4)
memory usage: 5.4+ MB


In [43]:
combine = lambda x: ','.join(x)
ON_biztips_df1 = ON_biztips_df.groupby('business_id').agg({'user_id': combine,'tip_text':combine,\
                                                           'tip_compliment_count':'sum','tip_date': combine}).reset_index()


In [44]:
ON_biztips_df1.head()

,business_id,user_id,tip_text,tip_compliment_count,tip_date
0,--6MefnULPED_I942VcFNA,"EiP1OFgs-XGcKZux0OKWIA,2oMkzQcRL7-d7URt3Xo_Xg,...","BBQ pork is sold out early on Saturday,$5 lunc...",0.0,"2013-03-24 01:12:20,2014-07-11 15:48:40,2014-0..."
1,--DaPTJW3-tB1vP-PfdTEg,"21akEyhDC8yETHMaVYaPKw,2fXZuNKP7Bo-yRrINVbVVA,...",Awesome. Huge portions definitely worth a vis...,0.0,"2014-04-20 16:03:20,2013-09-06 14:44:09,2017-0..."
2,--SrzpvFLwP_YFwB_Cetow,"G-6X-llgA_qAxGxocykHzQ,VPtIU8zEu8Iv-IAFnFWa0Q,...","Congee not very good...,This place is great fo...",0.0,"2017-03-14 15:14:36,2011-02-10 03:51:06,2012-1..."
3,--q6datkI-f0EoVheXNEeQ,"G-6X-llgA_qAxGxocykHzQ,CxDOIDnH8gp9KXzpBHJYXw","Street cars would still be operating, avoid st...",0.0,"2015-11-18 01:10:46,2013-01-03 14:19:31"
4,-0CCHBui57tZ_1y_14X-5Q,_NMdJpHLvmSnTo1KrtvRcg,"Amazing cocktails, beautiful decor, a lovely p...",0.0,2017-07-23 13:37:02


In [45]:
bizinfo_ON_tip_df = pd.merge(business_ON_checkin_df,ON_biztips_df1,on='business_id',how='left')

In [46]:
bizinfo_ON_tip_df.head()

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,name,postal_code,review_count,stars,state,checkin_date,user_id,tip_text,tip_compliment_count,tip_date
0,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"2010-07-05 16:56:31, 2010-12-21 20:30:01, 2010...","fENhF1F6I9c2ssnkfprYzw,RscJR5zqb3WL4eQezBX5rA,...","Spring & egg rolls are great, get them with ex...",1.0,"2016-08-08 01:48:14,2013-06-21 23:44:22,2017-1..."
1,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,The UPS Store,L5V 0B1,3,2.5,ON,"2014-03-31 21:46:17, 2015-01-27 19:05:03, 2018...",jnB_saJqNfOmVoCWquhAzg,Good service,0.0,2014-03-31 21:46:39
2,1455 16th Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,Hot Yoga Wellness,L4B 3G6,4,4.0,ON,"2016-04-19 17:38:29, 2016-04-22 01:14:54, 2016...","CxDOIDnH8gp9KXzpBHJYXw,5vRBvQEbR12jXzC50cBYqw","Replacing My Yoga Studio,Went there for the fi...",0.0,"2016-04-19 16:16:40,2017-09-20 15:10:00"
3,"445 King Street W, Suite 101",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Totum Life Science,M5V 1K4,23,4.0,ON,"2010-10-23 14:00:26, 2010-10-24 16:33:33, 2010...","n1bfatw5wT1eRYWmHSLo9g,gLSQCEhGy7-uRnNbNnVykg","One-week guest pass is $44.,Very average. Best...",0.0,"2014-08-07 03:35:10,2013-07-28 21:31:25"
4,"865 York Mills Road, Unit 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,Bakery Gateau,M3B 1Y6,8,4.5,ON,"2016-01-24 00:15:08, 2016-01-25 22:42:18, 2016...","keLUgL_4y60BkppiAsIk8Q,keLUgL_4y60BkppiAsIk8Q",Tried the tiramisu mini cake $16.. very light!...,1.0,"2016-03-22 20:21:20,2016-01-23 20:02:08"


In [47]:
ON_biz_photo_df = pd.merge(ON_business_id_df,photo_df,on='business_id',how='left')
ON_biz_photo_df.count()

business_id    68035
caption        18134
label          42054
photo_id       42054
dtype: int64

In [48]:
ON_biz_photo_df.drop_duplicates(inplace=True)
ON_biz_photo_df.fillna('null',inplace=True)
ON_biz_photo_df.count()


business_id    68016
caption        68016
label          68016
photo_id       68016
dtype: int64

In [49]:
ON_biz_photo_df = ON_biz_photo_df.groupby('business_id').agg({'photo_id':combine,'label':combine,'caption':combine}).reset_index()
ON_biz_photo_df.count()

business_id    33412
photo_id       33412
label          33412
caption        33412
dtype: int64

In [50]:
Ontario_bizinfo_df = pd.merge(bizinfo_ON_tip_df,ON_biz_photo_df,on='business_id',how='left')
Ontario_bizinfo_df.count()

address                                  33012
attributes.AcceptsInsurance                566
attributes.AgesAllowed                      16
attributes.Alcohol                       11208
attributes.Ambience                      11561
attributes.BYOB                              0
attributes.BYOBCorkage                       1
attributes.BestNights                      533
attributes.BikeParking                   17313
attributes.BusinessAcceptsBitcoin            2
attributes.BusinessAcceptsCreditCards      613
attributes.BusinessParking               21523
attributes.ByAppointmentOnly              5050
attributes.Caters                         9030
attributes.CoatCheck                       648
attributes.Corkage                           0
attributes.DietaryRestrictions              17
attributes.DogsAllowed                    1590
attributes.DriveThru                       552
attributes.GoodForDancing                  855
attributes.GoodForKids                   14726
attributes.Go

In [52]:
Ontario_bizinfo_df.to_csv('./DataSet/ontario_biz_data.csv')

In [ ]:
#Load into the PostgreSQL
engine = create_engine('postgresql+psycopg2://postgres:postgres@@127.0.0.1/yelp_data')
Ontario_bizinfo_df.to_sql('yelp_on_full_data',con=engine)

